In [1]:
!pip install nltk==3.8.1 sentence-transformers
# !pip install pytorch2tikz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b504ac372a76e36d217424b4798052c9ceeb426f9b8e7e91c707b80b98bdb2e3
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Input, Embedding, Dropout, Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score


In [7]:

df = pd.read_csv(r'../scraper/datasets/finalDataset/latest3.csv')

le = LabelEncoder()
cdf = df.copy()
cdf['truth_value'] = le.fit_transform(cdf['truth_value'])

In [8]:
nan_mask = cdf['simple_sentence'].isna()
nan_indices = cdf[nan_mask].index

# Replace NaN values in 'B' with values from column 'A'
cdf.loc[nan_indices, 'simple_sentence'] = df.loc[nan_indices, 'claim']

cdf['simple_sentence'].isna().sum()

0

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
def stemm_text(text):
    return ' '.join([stemmer.stem(w) for w in text.split(' ')])

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split(' ')])

T = cdf['claim'].str.split(' \n\n---\n\n').str[0]
T = T.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()
stop = stopwords.words('english')
T = T.apply(lambda x: ' '.join([y for y in x.split() if not y.isdigit()]))
T = T.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
cdf['claim'] = T

T = cdf['simple_sentence'].str.split(' \n\n---\n\n').str[0]
T = T.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()
stop = stopwords.words('english')
T = T.apply(lambda x: ' '.join([y for y in x.split() if not y.isdigit()]))
T = T.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
cdf['simple_sentence'] = T
cdf.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: The default value of regex will change from True to False in a future version.


,Unnamed: 0,claim,simple_sentence,truth_value,source
0,0,ukraine theft homicide levels rose due power o...,ukraine theft homicide levels rose due power o...,0,vox-ukraine
1,1,ukrainians beat two berlin residents speaking ...,ukrainians beat two berlin residents speaking ...,0,vox-ukraine
2,2,quote paul goebbels banderites,quote paul goebbels banderites,0,vox-ukraine
3,3,culture good neighborliness course ukrainian s...,culture good neighborliness course ukrainian s...,0,vox-ukraine
4,4,us research ukraine led increase incidence tic...,us research ukraine led increase incidence tic...,0,vox-ukraine
5,5,chile law rights mutants genetically modified ...,chile law rights mutants approved ostap stakhi...,0,vox-ukraine
6,6,covid incidence rate became zero late may,covid incidence rate became zero late may,0,vox-ukraine
7,7,risk death among children vaccinated covid tim...,risk death among children vaccinated covid tim...,0,vox-ukraine
8,8,russias army destroyed,russias army destroyed,0,politifact
9,9,war ukraine,war ukraine,0,politifact


In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
cdf = cdf.sample(frac=1).reset_index(drop=True)
X = cdf[['claim', 'simple_sentence']]
y = cdf['truth_value']

# X = X.apply(lambda w: lemmatize_text(w))
X['claim'] = X['claim'].apply(lambda w: lemmatize_text(w))
X['simple_sentence'] = X['simple_sentence'].apply(lambda w: lemmatize_text(w))
X.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,claim,simple_sentence
0,kyiv developed plan genocide population donbas,kyiv developed plan genocide population donbas
1,president joe biden said idea going send tank ...,president joe biden said idea going send tank ...
2,asymptomatic carrier coronavirus cannot infect...,asymptomatic carrier coronavirus cannot infect...
3,war ukraine sky kyiv come drone attack u poise...,war ukraine sky kyiv come drone attack u poise...
4,u troop ground ukraine,u troop ground ukraine


In [9]:
!mkdir /root/nltk_data

In [10]:
nltk.download('wordnet', download_dir='./wordnet')

[nltk_data] Downloading package wordnet to ./wordnet...


True

In [11]:
!unzip -o ./wordnet/corpora/wordnet.zip -d /root/nltk_data/corpora/ 

Archive:  ./wordnet/corpora/wordnet.zip
   creating: /root/nltk_data/corpora/wordnet/
  inflating: /root/nltk_data/corpora/wordnet/lexnames  
  inflating: /root/nltk_data/corpora/wordnet/data.verb  
  inflating: /root/nltk_data/corpora/wordnet/index.adv  
  inflating: /root/nltk_data/corpora/wordnet/adv.exc  
  inflating: /root/nltk_data/corpora/wordnet/index.verb  
  inflating: /root/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /root/nltk_data/corpora/wordnet/data.adj  
  inflating: /root/nltk_data/corpora/wordnet/index.adj  
  inflating: /root/nltk_data/corpora/wordnet/LICENSE  
  inflating: /root/nltk_data/corpora/wordnet/citation.bib  
  inflating: /root/nltk_data/corpora/wordnet/noun.exc  
  inflating: /root/nltk_data/corpora/wordnet/verb.exc  
  inflating: /root/nltk_data/corpora/wordnet/README  
  inflating: /root/nltk_data/corpora/wordnet/index.sense  
  inflating: /root/nltk_data/corpora/wordnet/data.noun  
  inflating: /root/nltk_data/corpora/wordnet/data.adv  
  infl

In [19]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:

# Define the model architecture
class BiLSTM(nn.Module):
    def __init__(self, num_words, embed_size, hidden_size, fc_out_size, output_size, dropout_rate):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(num_words, embed_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.bilstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)
#         self.fc = nn.Linear(hidden_size * 2, output_size)
        self.fc = nn.Linear(hidden_size * 2, fc_out_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.bilstm1(x)
        x = self.dropout(x)
        x, _ = self.bilstm2(x)
        x = self.dropout(x)
        x = self.fc(x[:, -1, :])
        return x
    


In [21]:

# Define the model architecture
class BiLSTM2(nn.Module):
    def __init__(self, num_words, embed_size, hidden_size, fc_out_size, output_size, dropout_rate):
        super(BiLSTM2, self).__init__()
        self.embedding = nn.Embedding(num_words, embed_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.bilstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)
#         self.fc = nn.Linear(hidden_size * 2, output_size)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.bilstm1(x)
        x = self.dropout(x)
        x, _ = self.bilstm2(x)
        x = self.dropout(x)
        x = self.fc(x[:, -1, :])
        return self.sigmoid(x)
    


In [43]:

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn import svm

class Trainer:
    def __init__(self):
        self.model1 = None
        self.model2 = None
        self.clf = None
        self.svmx_train = None
        self.svmy_train = None
        self.svmx_test = None
        self.svmy_test = None

    def train_model1(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata):
        te = 25
        self.model1 = BiLSTM2(num_words=2000, embed_size=60, hidden_size=64, fc_out_size=5000, output_size=1, dropout_rate=0.2).to(device)
        self.model1.train()
        criterion = nn.BCELoss().to(device)
        optimizer = optim.Adam(self.model1.parameters(), lr=0.01)

        self.model1.train()
        for epoch in range(te):
            optimizer.zero_grad()
            outputs = self.model1(train_claim.to(device))
            loss = criterion(outputs.squeeze(), y_traindata.to(device))
            loss.backward()
            optimizer.step()

        self.model1.eval()
        with torch.no_grad():
            predictions_tr = self.model1(train_claim.to(device))
            predictions_tr = (predictions_tr > 0.5).to('cpu').int().squeeze().numpy()
        with torch.no_grad():
            predictions_te = self.model1(test_claim)
            predictions_te = (predictions_te > 0.5).to('cpu').int().squeeze().numpy()

        print(f"Model 1, Total Epochs: {te}, Train Accuracy: {metrics.accuracy_score(y_traindata.to('cpu'), predictions_tr)} Test Accuracy: {metrics.accuracy_score(y_testdata.to('cpu'), predictions_te)}")
        
    
    def train_model2(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata):
        te = 25
        self.model2 = BiLSTM2(num_words=2000, embed_size=60, hidden_size=64, fc_out_size=5000, output_size=1, dropout_rate=0.2).to(device)
        self.model2.train()
        criterion = nn.BCELoss().to(device)
        optimizer = optim.Adam(self.model2.parameters(), lr=0.01)

        self.model2.train()
        for epoch in range(te):
            optimizer.zero_grad()
            outputs = self.model2(train_ss.to(device))
            loss = criterion(outputs.squeeze(), y_traindata.to(device))
            loss.backward()
            optimizer.step()

        self.model2.eval()
        with torch.no_grad():
            predictions_tr = self.model2(train_ss.to(device))
            predictions_tr = (predictions_tr > 0.5).to('cpu').int().squeeze().numpy()
        with torch.no_grad():
            predictions_te = self.model2(test_ss)
            predictions_te = (predictions_te > 0.5).to('cpu').int().squeeze().numpy()

        print(f"Model 2, Total Epochs: {te}, Train Accuracy: {metrics.accuracy_score(y_traindata.to('cpu'), predictions_tr)} Test Accuracy: {metrics.accuracy_score(y_testdata.to('cpu'), predictions_te)}")

        
    def report_stats_on_train_set_of_inidividual_models(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata):
        ytr = []
        cury = y_traindata.detach().cpu().numpy()
        xtr1 = []
        xtr2 = []
        for i in range(len(cury)):
            ytr.append(cury[i])
            xtr1.append(train_claim[i].detach().cpu().numpy())
            xtr2.append(train_ss[i].detach().cpu().numpy())

        self.model1.eval()
        self.model2.eval()
        with torch.no_grad():
            p1 = self.model1(torch.tensor(np.array(xtr1)).to(device))
            p1 = (p1 > 0.5).to('cpu').int().squeeze().numpy().tolist()
            p2 = self.model2(torch.tensor(np.array(xtr2)).to(device))
            p2 = (p2 > 0.5).to('cpu').int().squeeze().numpy().tolist()

            Y_TRAIN = torch.tensor(np.array(ytr))

            print("Model 1 results on train set")
            print(classification_report(Y_TRAIN, p1, digits=4))
            print(confusion_matrix(Y_TRAIN, p1))
            print("Model 2 results on train set")
            print(classification_report(Y_TRAIN, p2, digits=4))
            print(confusion_matrix(Y_TRAIN, p2))
            
    def report_stats_on_test_set_of_inidividual_models(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata):
        ytr = []
        cury = y_testdata.detach().cpu().numpy()
        xtr1 = []
        xtr2 = []
        for i in range(len(cury)):
            ytr.append(cury[i])
            xtr1.append(test_claim[i].detach().cpu().numpy())
            xtr2.append(test_ss[i].detach().cpu().numpy())

        self.model1.eval()
        self.model2.eval()
        with torch.no_grad():
            p1 = self.model1(torch.tensor(np.array(xtr1)).to(device))
            p1 = (p1 > 0.5).to('cpu').int().squeeze().numpy().tolist()
            p2 = self.model2(torch.tensor(np.array(xtr2)).to(device))
            p2 = (p2 > 0.5).to('cpu').int().squeeze().numpy().tolist()

            Y_TEST = torch.tensor(np.array(ytr))

            print("Model 1 results on test set")
            print(classification_report(Y_TEST, p1, digits=4))
            print(confusion_matrix(Y_TEST, p1))
            print("Model 2 results on test set")
            print(classification_report(Y_TEST, p2, digits=4))
            print(confusion_matrix(Y_TEST, p2))
            
            
    def train_test_svm(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata, w1, w2):
        self.svmx_train = []
        self.svmy_train = []
        self.svmx_test = []
        self.svmy_test = []

        ytr = []
        cury = y_traindata.detach().cpu().numpy()
        xtr1 = []
        xtr2 = []
        yte = []
        curyte = y_testdata.detach().cpu().numpy()
        xte1 = []
        xte2 = []
        for i in range(len(cury)):
            ytr.append(cury[i])
            xtr1.append(train_claim[i].detach().cpu().numpy())
            xtr2.append(train_ss[i].detach().cpu().numpy())
        for i in range(len(curyte)):
            yte.append(curyte[i])
            xte1.append(test_claim[i].detach().cpu().numpy())
            xte2.append(test_ss[i].detach().cpu().numpy())


        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output
            return hook

        self.model1.bilstm2.register_forward_hook(get_activation('bilstm2_m1'))
        self.model1.eval()
        self.model2.bilstm2.register_forward_hook(get_activation('bilstm2_m2'))
        self.model2.eval()
        with torch.no_grad():
            p1 = self.model1(torch.tensor(np.array(xtr1)).to(device))
            p1 = (p1 > 0.5).to('cpu').int().squeeze().numpy().tolist()
            p2 = self.model2(torch.tensor(np.array(xtr2)).to(device))
            p2 = (p2 > 0.5).to('cpu').int().squeeze().numpy().tolist()
            weighted_avg = (w1 * activation['bilstm2_m1'][0][:, -1, :].clone().detach()) + (w2 * activation['bilstm2_m2'][0][:, -1, :].clone().detach())
            self.svmx_train = weighted_avg.detach().cpu().numpy()
            self.svmy_train = ytr

            p1 = self.model1(torch.tensor(np.array(xte1)).to(device))
            p1 = (p1 > 0.5).to('cpu').int().squeeze().numpy().tolist()
            p2 = self.model2(torch.tensor(np.array(xte2)).to(device))
            p2 = (p2 > 0.5).to('cpu').int().squeeze().numpy().tolist()
            weighted_avg = (w1 * activation['bilstm2_m1'][0][:, -1, :].clone().detach()) + (w2* activation['bilstm2_m2'][0][:, -1, :].clone().detach())
            self.svmx_test = weighted_avg.detach().cpu().numpy()
            self.svmy_test = yte

        self.clf = svm.SVC(probability=True, kernel='poly')
        self.clf.fit(self.svmx_train, self.svmy_train)

        
    def report_stats_on_train_set_of_svm(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata):
        preds = self.clf.predict(self.svmx_train)
        print(classification_report(self.svmy_train, preds, digits=4))
        print(confusion_matrix(self.svmy_train, preds))
        
    def report_stats_on_test_set_of_svm(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata):
        preds = self.clf.predict(self.svmx_test)
        print(classification_report(self.svmy_test, preds, digits=4))
        print(confusion_matrix(self.svmy_test, preds))
        
    def train_all(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata, w1, w2):
        self.train_model1(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata)
        self.train_model2(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata)
        self.train_test_svm(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata, w1, w2)
        
    def test_all(self, train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata):
        print('Model 1 and 2 results on train set:')
        self.report_stats_on_train_set_of_inidividual_models(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata)
        print('Model 1 and 2 results on test set:')
        self.report_stats_on_test_set_of_inidividual_models(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata)
        print('SVM results on train set:')
        self.report_stats_on_train_set_of_svm(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata)
        print('SVM results on test set:')
        self.report_stats_on_test_set_of_svm(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata)
        
    def prepare_data(self, claim, ss, truths):
        xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
        
        tk1 = text.Tokenizer(num_words=2000)
        tk2 = text.Tokenizer(num_words=2000)
        tk1.fit_on_texts(claim)
        tk2.fit_on_texts(ss)
        tokenized_train_claim = tk1.texts_to_sequences(claim)
        tokenized_train_ss = tk2.texts_to_sequences(ss)
        X_claim = torch.tensor(sequence.pad_sequences(tokenized_train_claim, maxlen=60)).to(device)
        X_ss = torch.tensor(sequence.pad_sequences(tokenized_train_ss, maxlen=60)).to(device)

        y_labeldata = torch.tensor(truths.values).float().to(device)

        return X_claim, X_ss, y_labeldata
    
    

k = 5
kf = KFold(n_splits=k, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print( '\n\n', 'Fold:', fold)
    trainer = Trainer()
    X_claim, X_ss, y_labeldata = trainer.prepare_data(X['claim'], X['simple_sentence'], y)
    
    train_claim = X_claim[train_idx]
    train_ss = X_claim[train_idx]
    test_claim = X_claim[val_idx]
    test_ss = X_claim[val_idx]
    y_traindata = y_labeldata[train_idx]
    y_testdata = y_labeldata[val_idx]
    
    trainer.train_all(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata, 0.75, 0.25)
    trainer.test_all(train_claim, test_claim, train_ss, test_ss, y_traindata, y_testdata)



 Fold: 0
Model 1, Total Epochs: 25, Train Accuracy: 0.9364437532400207 Test Accuracy: 0.8648424543946932
Model 2, Total Epochs: 25, Train Accuracy: 0.9318818040435459 Test Accuracy: 0.8756218905472637
Model 1 and 2 results on train set:
Model 1 results on train set
              precision    recall  f1-score   support

         0.0     0.9325    0.9478    0.9401      5075
         1.0     0.9409    0.9239    0.9323      4570

    accuracy                         0.9364      9645
   macro avg     0.9367    0.9358    0.9362      9645
weighted avg     0.9365    0.9364    0.9364      9645

[[4810  265]
 [ 348 4222]]
Model 2 results on train set
              precision    recall  f1-score   support

         0.0     0.9293    0.9423    0.9357      5075
         1.0     0.9349    0.9204    0.9276      4570

    accuracy                         0.9319      9645
   macro avg     0.9321    0.9313    0.9316      9645
weighted avg     0.9319    0.9319    0.9319      9645

[[4782  293]
 [ 364 42